In [14]:
import numpy as np
import os
import xarray as xr # With h5netcdf dependency

In [15]:
# Regrids NSIDC Polar Pathfinder Sea Ice Velocities from EASE to regular lat lon coordinate system
# NOTE See Southern Hemisphere vs Northern Hemisphere vector rotation below

# 04SEP24 Northern Hemisphere Version

# Enter data source path
PATH_SOURCE = "/home/jbassham/north/data/motion"
# Enter file name (end of URL) with placeholder {year}
FNAM = "icemotion_daily_nh_25km_{year}0101_{year}1231_v4.1.nc"

# Enter destination path
PATH_DEST = "/home/jbassham/north/data"

# Enter years to regrid
START_YEAR = 2020
END_YEAR = 2020

# Enter bounds for lat and lon
LAT_LIM = [60, 90] 
LON_LIM = [-180, 180]

# Enter new grid resolution (converting 25 km to degrees latitude)
YRES = 25 / 111   # Degrees latitude at 25km resolution
XRES = 25 / (111*np.cos(np.abs((LAT_LIM[0]+LAT_LIM[1])/2))) # Degrees longitude at 25km resolution (based on average latitude)

In [18]:
def main():
    # Create arrays for new lat and lon grid
    lat = np.arange(LAT_LIM[0], LAT_LIM[1] + YRES) # Latitude
    lon = np.arange(LON_LIM[0], LON_LIM[1] + XRES) # Longitude
    nlat = len(lat)
    nlon = len(lon)
    
    # define years to process
    do_years = np.arange(START_YEAR, END_YEAR + 1)

    # Intialize interpolation with one arbitrary grid from the original data
    filename = FNAM.format(year = END_YEAR)
    path = os.path.join(PATH_SOURCE, filename)

    with xr.open_dataset(path) as data:
        lat_ease = data['latitude'].values
        lon_ease = data['longitude'].values

        # NOTE Cropping might not be needed (fast without) - buggy with crop need to also crop when looking for indicies
        # # Find indices for crop within limits lat and lon bounds
        # # * Extract indices along [0]th dimension
        # ix_crop = np.unique(np.where((data['latitude'].values >= LAT_LIM[0]) & (data['latitude'].values <= LAT_LIM[1]) & (data['longitude'].values >= LON_LIM[0]) & (data['longitude'].values <= LON_LIM[1]))[0])
        # # * Extract indices along [1]th dimension
        # iy_crop = np.unique(np.where((data['latitude'].values >= LAT_LIM[0]) & (data['latitude'].values <= LAT_LIM[1]) & (data['longitude'].values >= LON_LIM[0]) & (data['longitude'].values <= LON_LIM[1]))[1])


    # Initialize arrays for interpolation indices
    jj = np.zeros((nlat,nlon), dtype=int)
    ii = np.zeros((nlat,nlon), dtype=int)

    # Iterate through new grid's lat and lon points
    for j in range(nlat):
        for i in range(nlon):

            # Find absolute value distances of j'th lat from entire lat_ease array and store in array
            dy = (lat[j]-lat_ease)**2

            # Find absolute value distances of i'th lat from entire lat_ease array and store in array  
            dx = (lon[i]-lon_ease)**2
        
            # Find distances (we don't need sqrt here b/c not using actual value, just minimum)
            ds = dx + dy

            # Find indices of minimum ds value
            i_neighbors = np.where(ds == np.min(ds))

            # Take minium of lat and lon indices (lower left corner) for consistency, store in array
            jj[j,i] = np.min(i_neighbors[0])
            ii[j,i] = np.min(i_neighbors[1])

    # Initialize lists to append data arrays from each year
    u_append = []
    v_append = []
    error_append = []
    t_append = []


    # Iterate through years
    for year in do_years:
        # Create path for each year's file
        filename = FNAM.format(year = year)
        path = os.path.join(PATH_SOURCE, filename)

        # Open dataset as xarray
        with xr.open_dataset(path) as data:
            # NOTE Cropping might not be needed
            # Crop data within bounds 
            # data = data.isel(x = ix_crop, y = iy_crop)
            u_ease = data['u'].values                             # Sea ice x velocity (t, y, x), cm/s
            v_ease = data['v'].values                             # Sea ice y velocity (t, y, x), cm/s 
            error_ease = data['icemotion_error_estimate'].values  # Ice motion error estimates
            time = data['time']                                   # time (days since 01,01,1970)
            nt = len(time)

        # Initialize data arrays for current year
        dims = (nt, nlat, nlon)
        u = np.zeros(dims) # zonal ice velocity
        v = np.zeros(dims) # meridional ice velocity
        error = np.zeros(dims) # icemotion error estimates

        for i in range(nlon):
            for j in range(nlat):
                iii = ii[j,i]
                jjj = jj[j,i]
                
                # SOUTHERN HEMISPHERE vector rotation to East/ North
                # u[:,j, i] = u_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj, iii])) - v_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj,iii]))
                # v[:,j, i] = u_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj, iii])) + v_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj,iii]))

                # NORTHERN HEMISPHERE vector rotation to East/ North
                u[:,j, i] = u_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj, iii])) + v_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj,iii]))
                v[:,j, i] = -u_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj, iii])) + v_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj,iii]))

                error[:,j,i] = error_ease[:,jjj,iii]
            print(f"lon {i} complete")

        # Append data to list of years
        u_append.append(u)
        v_append.append(v)
        error_append.append(error)
        t_append.append(time)

        # Year complete
        print(f"Year {year} Regrid")

    # Concatenate list of yearly data arrays along time dimension
    u_concat = np.concatenate(u_append, axis = 0)
    v_concat = np.concatenate(v_append, axis = 0)
    error_concat = np.concatenate(error_append, axis = 0)

    # Save time series data as npz variables
    fnam = f"/motion_ppv4_latlon_nh_{START_YEAR}_{END_YEAR}"
    np.savez_compressed(PATH_DEST + fnam, allow_pickle = True, u = u_concat, v = v_concat, error = error_concat, time = t_append, lat = lat, lon = lon)
    print(f"Variables Saved at path {PATH_DEST + fnam}.npz")

    return  

if __name__ == "__main__":
    main()

lon 0 complete
lon 1 complete
lon 2 complete
lon 3 complete
lon 4 complete
lon 5 complete
lon 6 complete
lon 7 complete
lon 8 complete
lon 9 complete
lon 10 complete
lon 11 complete
lon 12 complete
lon 13 complete
lon 14 complete
lon 15 complete
lon 16 complete
lon 17 complete
lon 18 complete
lon 19 complete
lon 20 complete
lon 21 complete
lon 22 complete
lon 23 complete
lon 24 complete
lon 25 complete
lon 26 complete
lon 27 complete
lon 28 complete
lon 29 complete
lon 30 complete
lon 31 complete
lon 32 complete
lon 33 complete
lon 34 complete
lon 35 complete
lon 36 complete
lon 37 complete
lon 38 complete
lon 39 complete
lon 40 complete
lon 41 complete
lon 42 complete
lon 43 complete
lon 44 complete
lon 45 complete
lon 46 complete
lon 47 complete
lon 48 complete
lon 49 complete
lon 50 complete
lon 51 complete
lon 52 complete
lon 53 complete
lon 54 complete
lon 55 complete
lon 56 complete
lon 57 complete
lon 58 complete
lon 59 complete
lon 60 complete
lon 61 complete
lon 62 complete
lo

In [13]:
data = np.load('/home/jbassham/north/data/motion_ppv4_latlon_nh_1986_2020.npz')
ui = data['u']
vi = data['v']
errori = data['error']
lat = data['lat']
lon = data['lon']
time = data['time']

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [ ]:
### Saving .npz for each year to load in script later ###
# def main():
# Create arrays for new lat and lon grid
lat = np.arange(LAT_LIM[0], LAT_LIM[1] + YRES) # Latitude
lon = np.arange(LON_LIM[0], LON_LIM[1] + XRES) # Longitude
nlat = len(lat)
nlon = len(lon)

# define years to process
do_years = np.arange(START_YEAR, END_YEAR + 1)

# Intialize interpolation with one arbitrary grid from the original data
filename = FNAM.format(year = END_YEAR)
path = os.path.join(PATH_SOURCE, filename)

with xr.open_dataset(path) as data:
    lat_ease = data['latitude'].values
    lon_ease = data['longitude'].values

    # NOTE Cropping might not be needed (fast without) - buggy with crop need to also crop when looking for indicies
    # # Find indices for crop within limits lat and lon bounds
    # # * Extract indices along [0]th dimension
    # ix_crop = np.unique(np.where((data['latitude'].values >= LAT_LIM[0]) & (data['latitude'].values <= LAT_LIM[1]) & (data['longitude'].values >= LON_LIM[0]) & (data['longitude'].values <= LON_LIM[1]))[0])
    # # * Extract indices along [1]th dimension
    # iy_crop = np.unique(np.where((data['latitude'].values >= LAT_LIM[0]) & (data['latitude'].values <= LAT_LIM[1]) & (data['longitude'].values >= LON_LIM[0]) & (data['longitude'].values <= LON_LIM[1]))[1])


# Initialize arrays for interpolation indices
jj = np.zeros((nlat,nlon), dtype=int)
ii = np.zeros((nlat,nlon), dtype=int)

# Iterate through new grid's lat and lon points
for j in range(nlat):
    for i in range(nlon):

        # Find absolute value distances of j'th lat from entire lat_ease array and store in array
        dy = (lat[j]-lat_ease)**2

        # Find absolute value distances of i'th lat from entire lat_ease array and store in array  
        dx = (lon[i]-lon_ease)**2

        # Find distances (we don't need sqrt here b/c not using actual value, just minimum)
        ds = dx + dy

        # Find indices of minimum ds value
        i_neighbors = np.where(ds == np.min(ds))

        # Take minium of lat and lon indices (lower left corner) for consistency, store in array
        jj[j,i] = np.min(i_neighbors[0])
        ii[j,i] = np.min(i_neighbors[1])


# Iterate through years
for year in do_years:
    # Create path for each year's file
    filename = FNAM.format(year = year)
    path = os.path.join(PATH_SOURCE, filename)

    # Open dataset as xarray
    with xr.open_dataset(path) as data:
        # Crop data within bounds 
#       data = data.isel(x = ix_crop, y = iy_crop)
        u_ease = data['u'].values                      # Sea ice x velocity (t, y, x), cm/s
        v_ease = data['v'].values                      # Sea ice y velocity (t, y, x), cm/s 
        error_ease = data['icemotion_error_estimate'].values  # Ice motion error estimates
        time = data['time']                            # time (days since 01,01,1970)
        nt = len(time)

    # Initialize data arrays for current year
    dims = (nt, nlat, nlon)
    u = np.zeros(dims) # zonal ice velocity
    v = np.zeros(dims) # meridional ice velocity
    error = np.zeros(dims) # icemotion error estimates

    for i in range(nlon):
        for j in range(nlat):
            iii = ii[j,i]
            jjj = jj[j,i]
    
            # # SOUTHERN HEMISPHERE vector rotation to East/ North
            # u[:,j, i] = u_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj, iii])) - v_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj,iii]))
            # v[:,j, i] = u_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj, iii])) + v_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj,iii]))
        
            # NORTHERN HEMISPHERE vector rotation to East/ North
            u[:,j, i] = u_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj, iii])) + v_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj,iii]))
            v[:,j, i] = -u_ease[:,jjj, iii]*np.sin(np.radians(lon_ease[jjj, iii])) + v_ease[:,jjj, iii]*np.cos(np.radians(lon_ease[jjj,iii]))

            error[:,j,i] = error_ease[:,jjj,iii]
            
        print(f"Lon {i} complete")

    # Year complete
    print(f"Year {year} Regrid")

    # Save time series data as npz variables
    fnam = f"motion_ppv4_latlon_nh_{year}"
    np.savez_compressed(os.path.join(PATH_DEST, fnam), u = u, v = v, error = error, time = time, lat = lat, lon = lon)
    
    print(f"Variables Saved at path {PATH_DEST + fnam}.npz")

# if __name__ == "__main__":
#     main()